# Import Libraries

In [18]:
import gensim
import spacy
import re
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, SimpleRNN, GRU
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

# Load Datasets

In [3]:
train = pd.read_csv('/content/drive/MyDrive/nlp_project/data_cleaning/preprocessed_train.csv')
train.drop(columns='Unnamed: 0', inplace=True)
train

,text,label
0,فين الذكور يا رجالة,LB
1,هه انت تشجع فنادي مش حاضرهم شايلين شامبيون اسي...,LY
2,فكرة حلوة و ممكن رسالة ع الموبايل,EG
3,ياا يالميدان كنت فين من زماان يارب كملها على خ...,EG
4,هما اعلامي التوك شو راحوا فين كله اعادة او مش ...,EG
...,...,...
118175,والله تذكرت يوم مات الحسن التاني كان من اول ال...,MA
118176,ممكن بس حد يدلني هو محماا علي ده ازاي حصل علي ...,EG
118177,ساكت ليه يا مجدي قوول وفضفض,EG
118178,سامعة الصراخ اللي طالع من جوه متخافيش دول صحاب...,EG


In [4]:
test = pd.read_csv('/content/drive/MyDrive/nlp_project/data_cleaning/preprocessed_test.csv')
test.drop(columns='Unnamed: 0', inplace=True)
test

,text,label
0,احنا بيقنا الصبح استاذ مجدي يومك بيضحك,EG
1,يا مشحبطيني يا اني,LB
2,زي النهارده السادات كان يشعر بالحرب مع مبارك و...,EG
3,عطاهم عصيير في كاس كبيير,MA
4,ولا ما سافل وحقير الا انتم عايزين الراجل يتسجن...,EG
...,...,...
29540,بعلمك فيه أزمة بعد كام ساعة بيتفقوا شوو هالمسخ...,LB
29541,حاجات ممكن تغيب عنك بسيطة وسهلة معناها لكن باز...,LY
29542,وبعدين ليا هلبة وقت ممشيتش لهون,LY
29543,فنان هايل وكوميدي من الدرجة الاولي وطبيعي جدا ...,EG


In [5]:
df = pd.concat([train, test], axis=0)
df

,text,label
0,فين الذكور يا رجالة,LB
1,هه انت تشجع فنادي مش حاضرهم شايلين شامبيون اسي...,LY
2,فكرة حلوة و ممكن رسالة ع الموبايل,EG
3,ياا يالميدان كنت فين من زماان يارب كملها على خ...,EG
4,هما اعلامي التوك شو راحوا فين كله اعادة او مش ...,EG
...,...,...
29540,بعلمك فيه أزمة بعد كام ساعة بيتفقوا شوو هالمسخ...,LB
29541,حاجات ممكن تغيب عنك بسيطة وسهلة معناها لكن باز...,LY
29542,وبعدين ليا هلبة وقت ممشيتش لهون,LY
29543,فنان هايل وكوميدي من الدرجة الاولي وطبيعي جدا ...,EG


# Load AraVec Pre-trained Model

In [6]:
# load the AraVec model
model = gensim.models.Word2Vec.load("/content/drive/MyDrive/nlp_project/full_grams_cbow_300_twitter.mdl")
print("We've",len(model.wv.index_to_key),"vocabularies")

We've 1476715 vocabularies


In [ ]:
# load AraVec Spacy model
nlp = spacy.load("/content/drive/MyDrive/nlp_project/spacyModel/spacy.aravec.model")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


# Tokenization

In [29]:
# Convert all entries in the 'text' column to strings
train['text'] = train['text'].astype(str)
test['text'] = test['text'].astype(str)

# Initialize and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['text'])
word_index = tokenizer.word_index

# Convert text data to sequences of indices
train_sequences = tokenizer.texts_to_sequences(train['text'])
test_sequences = tokenizer.texts_to_sequences(test['text'])

# Adjust indices to start from 1
for sequences in [train_sequences, test_sequences]:
    for sequence in sequences:
        for i in range(len(sequence)):
            sequence[i] += 1

In [ ]:
word_index

{'من': 1,
 'في': 2,
 'ما': 3,
 'و': 4,
 'مش': 5,
 'اللي': 6,
 'يا': 7,
 'بس': 8,
 'على': 9,
 'انا': 10,
 'الله': 11,
 'كل': 12,
 'ده': 13,
 'ولا': 14,
 'لو': 15,
 'علي': 16,
 'لا': 17,
 'كان': 18,
 'والله': 19,
 'انت': 20,
 'هو': 21,
 'دي': 22,
 'مع': 23,
 'هه': 24,
 'ان': 25,
 'فى': 26,
 'الناس': 27,
 'ايه': 28,
 'شي': 29,
 'عشان': 30,
 'لما': 31,
 'فيه': 32,
 'غير': 33,
 'ف': 34,
 'ع': 35,
 'عن': 36,
 'واحد': 37,
 'حد': 38,
 'يعني': 39,
 'او': 40,
 'عم': 41,
 'كده': 42,
 'مصر': 43,
 'اللى': 44,
 'فيها': 45,
 'بعد': 46,
 'دا': 47,
 'زي': 48,
 'حتى': 49,
 'ربنا': 50,
 'اي': 51,
 'ليه': 52,
 'احنا': 53,
 'لي': 54,
 'لكن': 55,
 'مين': 56,
 'ناس': 57,
 'شو': 58,
 'حاجة': 59,
 'ممكن': 60,
 'يوم': 61,
 'عليه': 62,
 'لازم': 63,
 'كتير': 64,
 'عليك': 65,
 'يارب': 66,
 'كنت': 67,
 'قبل': 68,
 'هي': 69,
 'كمان': 70,
 'كيف': 71,
 'أنا': 72,
 'انو': 73,
 'انه': 74,
 'الي': 75,
 'حاجه': 76,
 'هذا': 77,
 'وين': 78,
 'ربي': 79,
 'عارف': 80,
 'يلي': 81,
 'الا': 82,
 'علشان': 83,
 'اذا': 84,
 'جدا': 8

# Embedding Matrix by AraVec

In [30]:
# Create an embedding matrix
embedding_dim = 300  # This should match the dimension of your word vectors
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
    else:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = np.zeros(embedding_dim)

In [11]:
embedding_matrix.shape

(202305, 300)

# Finding max length for padding

In [ ]:
max_sequence_len = 0
for sentence in train_sequences:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

67


In [ ]:
max_sequence_len = 0
for sentence in test_sequences:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

61


In [31]:
max_sequence_length = 67
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

# Label Encoding

In [32]:
encoder = LabelEncoder()
y_encoded_train = encoder.fit_transform(train['label'])
y_encoded_train = to_categorical(y_encoded_train)
y_encoded_test = encoder.fit_transform(test['label'])
y_encoded_test = to_categorical(y_encoded_test)

In [33]:
num_words = len(tokenizer.word_index)
num_words

202304

# Building GRU Model

In [34]:
input_dim = len(tokenizer.word_index) + 1
input_dim

202305

In [35]:
embedding_matrix.shape

(202305, 300)

In [36]:
gru_bi = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim,  weights=[embedding_matrix],input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [37]:
gru_bi.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Train GRU Model

In [38]:
history = gru_bi.fit(train_padded, y_encoded_train, epochs=10, batch_size=128,
                    validation_data=(test_padded, y_encoded_test),
                    validation_steps=30)

Epoch 1/10
924/924 [==============================] - 78s 74ms/step - loss: 0.7782 - accuracy: 0.7084 - val_loss: 0.6028 - val_accuracy: 0.7888
Epoch 2/10
924/924 [==============================] - 39s 43ms/step - loss: 0.3503 - accuracy: 0.8828 - val_loss: 0.5708 - val_accuracy: 0.8016
Epoch 3/10
924/924 [==============================] - 36s 39ms/step - loss: 0.1503 - accuracy: 0.9520 - val_loss: 0.6706 - val_accuracy: 0.8076
Epoch 4/10
924/924 [==============================] - 34s 37ms/step - loss: 0.0787 - accuracy: 0.9755 - val_loss: 0.8114 - val_accuracy: 0.7971
Epoch 5/10
924/924 [==============================] - 35s 37ms/step - loss: 0.0524 - accuracy: 0.9834 - val_loss: 0.9039 - val_accuracy: 0.7987
Epoch 6/10
924/924 [==============================] - 32s 35ms/step - loss: 0.0384 - accuracy: 0.9874 - val_loss: 1.0347 - val_accuracy: 0.7930
Epoch 7/10
924/924 [==============================] - 33s 36ms/step - loss: 0.0302 - accuracy: 0.9900 - val_loss: 1.1578 - val_accuracy:

# Evaluate GRU Model

In [39]:
evaluation = gru_bi.evaluate(test_padded, y_encoded_test)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

924/924 [==============================] - 7s 8ms/step - loss: 1.2766 - accuracy: 0.7945
Test Loss: 1.2766005992889404
Test Accuracy: 0.7944830060005188


# Building LSTM Model

In [40]:
lstm_bi = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(num_words, 64, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [41]:
lstm_bi.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Train LSTM Model

In [42]:
history = lstm_bi.fit(train_padded, y_encoded_train, epochs=10, batch_size=128,
                    validation_data=(test_padded, y_encoded_test),
                    validation_steps=30)

Epoch 1/10
924/924 [==============================] - 74s 76ms/step - loss: 0.6931 - accuracy: 0.7452 - val_loss: 0.5105 - val_accuracy: 0.8271
Epoch 2/10
924/924 [==============================] - 21s 23ms/step - loss: 0.2558 - accuracy: 0.9166 - val_loss: 0.5137 - val_accuracy: 0.8284
Epoch 3/10
924/924 [==============================] - 19s 20ms/step - loss: 0.1096 - accuracy: 0.9666 - val_loss: 0.6525 - val_accuracy: 0.8216
Epoch 4/10
924/924 [==============================] - 14s 15ms/step - loss: 0.0627 - accuracy: 0.9806 - val_loss: 0.7614 - val_accuracy: 0.8135
Epoch 5/10
924/924 [==============================] - 15s 16ms/step - loss: 0.0417 - accuracy: 0.9867 - val_loss: 0.8319 - val_accuracy: 0.8109
Epoch 6/10
924/924 [==============================] - 15s 16ms/step - loss: 0.0324 - accuracy: 0.9896 - val_loss: 0.9879 - val_accuracy: 0.8003
Epoch 7/10
924/924 [==============================] - 15s 16ms/step - loss: 0.0263 - accuracy: 0.9917 - val_loss: 1.0314 - val_accuracy:

# Evaluate LSTM Model

In [43]:
evaluation = lstm_bi.evaluate(test_padded, y_encoded_test)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

924/924 [==============================] - 5s 5ms/step - loss: 1.2208 - accuracy: 0.7929
Test Loss: 1.220759630203247
Test Accuracy: 0.7928922176361084
